In [ ]:
dataset_name = 'monuseg'
device = 'cuda'
split_num = 1

import torch
import os
import matplotlib.pyplot as plt
import numpy as np
from Netlib.CenterMasklib.centermask import config
from Netlib.CenterMasklib.centermask.train_net.tools.dataset import CenterMaksMoNuSegDS
from Netlib.CDNetlib.tools.augmentation import MyCenterCrop
from Netlib.CenterMasklib.CentermaskInfer import CenteMaskInfer
arg = config.CenterMask50MoNuSeg_3FPN3_GN(ds=dataset_name,device=device,ds_num=split_num)
data = CenterMaksMoNuSegDS(arg.DATASET_DIR,[MyCenterCrop((256,256))],train=False)
model = CenteMaskInfer(arg)
preds =  np.zeros((0,256,256,2),dtype=np.int32)

for i in range(len(data)):
    
    img = data[i][0]['image'].unsqueeze(0).to(device)
    pred = model(img)
    pred = pred.astype(np.int32)
    pred[...,1] = (pred[...,1]!=0).astype(np.int32)
    #print(pred.shape,len(np.bincount(pred[...,0].flatten())))
    preds = np.concatenate((preds,pred),axis=0)

pred_path = os.path.join('numpy_result',arg.TEST_RESULT_DIR)+'/gts.npy'
gts_path = os.path.join('numpy_result',arg.TEST_RESULT_DIR)+'/preds.npy'
if not os.path.exists(os.path.dirname(pred_path)):
        os.makedirs(os.path.dirname(pred_path))
if not os.path.exists(os.path.dirname(gts_path)):
        os.makedirs(os.path.dirname(gts_path))

from Netlib.HoVerNetlib.tools.utils import draw_instance_map
from Netlib.HoVerNetlib.tools.metric import dice_score
def print_metric(pred_path, true_path):
        pred =np.load(pred_path)
        target = np.load(true_path)
        pred = torch.from_numpy(pred.astype(np.int16)).long() # [N, 256, 256, 2]
        target = torch.from_numpy(target.astype(np.int16)).long() # [N, 256, 256, 2]
        dice = dice_score(pred[..., 1], target[..., 1])
        print(f"dice score:{dice:.6f}")

np.save(pred_path,preds)
cp = MyCenterCrop((256,256))

gts = cp((data.labels))[0].permute(0,2,3,1).cpu().numpy()
np.save(gts_path,gts)
imgs = cp((data.imgs))[0]

if not os.path.exists(arg.figure_dir+'/preds'):
        print("not find the path")
        os.makedirs(arg.figure_dir+'/preds')

draw_instance_map(imgs,preds,fp=arg.figure_dir+'/preds')
print_metric(pred_path,gts_path)


print(pred_path)
print(gts_path)
print(arg.CLASSES-1)
os.system(f'python compute_metric/compute_stats.py --mode=seg_class --pred={pred_path} --true={gts_path} --class_num={arg.CLASSES-1}')